In [10]:
from superpowered import real_time_query, init
from bs4 import BeautifulSoup
import os
import requests

In [ ]:
# set API keys
os.environ["SUPERPOWERED_API_KEY_ID"] = "YOUR_API_KEY_ID"
os.environ["SUPERPOWERED_API_KEY_SECRET"] = "YOUR_API_KEY_SECRET"

In [ ]:
url = "https://en.wikipedia.org/wiki/Mont_Blanc"

# scrape the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
body = soup.find('body')
content = body.text

print (content)

In [16]:
init(api_key_id=os.environ["SUPERPOWERED_API_KEY_ID"], api_key_secret=os.environ["SUPERPOWERED_API_KEY_SECRET"])
query = "What is the name of the highest mountain in the Alps?"
response = real_time_query(query=query, passages=[content], top_k=5, summarize_results=True)
print (response["summary"])
print ()
print (response["results"])

Mont Blanc [2][3] is the highest mountain in the Alps and Western Europe, standing at 4,807.81 meters (15,774 feet) above sea level. It is located on the French-Italian border and is part of the Graian Alps range. Mont Blanc is the second-most prominent mountain in Europe, after Mount Elbrus, but the latter is considered part of the Eurasian land mass, which culminates at Mount Everest. The first ascent of Mont Blanc was made on August 8, 1786, by Jacques Balmat and Michel-Gabriel Paccard.

[{'content': 'fileSpecial pagesPermanent linkPage informationCite this pageWikidata item\n\n\n\n\n\t\tPrint/export\n\t\n\nDownload as PDFPrintable version\n\n\n\n\n\t\tIn other projects\n\t\n\nWikimedia CommonsWikivoyage\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFrom Wikipedia, the free encyclopedia\n\n\nHighest mountain in the Alps (4,808 m)\nThis article is about the mountain. For other uses, see Mont Blanc (disambiguation).\n\n\nMont BlancSummit of Mont Blanc and the Bosses ridgeHighest\xa0pointEl